# Scratchwork

In [1]:
from finite_algebras import make_finite_algebra, Examples, Group

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = Examples(alg_dir)

MAKE FINITE ALGEBRA: All inputs checked.
Constructing a Semigroup without checking inputs, <Group:A4, ID:140239155070160>
Constructing a Monoid without checking inputs, <Group:A4, ID:140239155070160>
Constructing a Group without checking inputs, <Group:A4, ID:140239155070160>
MAKE FINITE ALGEBRA: All inputs checked.
Constructing a Semigroup without checking inputs, <Group:D3, ID:140239155070032>
Constructing a Monoid without checking inputs, <Group:D3, ID:140239155070032>
Constructing a Group without checking inputs, <Group:D3, ID:140239155070032>
MAKE FINITE ALGEBRA: All inputs checked.
Constructing a Semigroup without checking inputs, <Group:D4, ID:140239155072272>
Constructing a Monoid without checking inputs, <Group:D4, ID:140239155072272>
Constructing a Group without checking inputs, <Group:D4, ID:140239155072272>
MAKE FINITE ALGEBRA: All inputs checked.
Constructing a Semigroup without checking inputs, <Group:Pinter29, ID:140239155072336>
Constructing a Monoid without checking in

In [4]:
a4 = ex.get_example(0)

In [5]:
a4

Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

In [6]:
id(a4)

140239155070160

In [7]:
print(a4)

<Group:A4, ID:140239155070160>


In [8]:
Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]],
check_inputs=True)

Constructing a Semigroup without checking inputs, <Group:A4, ID:140239155262544>
Constructing a Monoid without checking inputs, <Group:A4, ID:140239155262544>
Group: Checking inputs, <Group:A4, ID:140239155262544>


Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

In [9]:
Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

Constructing a Semigroup without checking inputs, <Group:A4, ID:140239155202832>
Constructing a Monoid without checking inputs, <Group:A4, ID:140239155202832>
Group: Checking inputs, <Group:A4, ID:140239155202832>


Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)

In [10]:
make_finite_algebra(
    'A4',
    'Alternating group on 4 letters (AKA Tetrahedral group)',
    ['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]],
)

MAKE FINITE ALGEBRA: All inputs checked.
Constructing a Semigroup without checking inputs, <Group:A4, ID:140239155217360>
Constructing a Monoid without checking inputs, <Group:A4, ID:140239155217360>
Constructing a Group without checking inputs, <Group:A4, ID:140239155217360>


Group(
'A4',
'Alternating group on 4 letters (AKA Tetrahedral group)',
['()', '(1 3 2)', '(1 2 3)', '(0 1)(2 3)', '(0 1 2)', '(0 1 3)', '(0 3 1)', '(0 3 2)', '(0 3)(1 2)', '(0 2 3)', '(0 2 1)', '(0 2)(1 3)'],
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4], [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7], [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8], [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1], [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10], [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9], [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2], [8, 6, 7, 11, 9, 10, 1, 2, 0, 4, 5, 3], [9, 10, 11, 4, 5, 3, 2, 0, 1, 7, 8, 6], [10, 11, 9, 2, 0, 1, 8, 6, 7, 3, 4, 5], [11, 9, 10, 8, 6, 7, 4, 5, 3, 1, 2, 0]]
)